<a href="https://colab.research.google.com/github/Taube0/MVP---Income-prediction/blob/main/House_income.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1º - Carga do Dataset com URL


In [1]:
# Importando bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.combine import SMOTEENN
import joblib

# Carregando o dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
           'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
           'hours-per-week', 'native-country', 'income']
data = pd.read_csv(url, header=None, names=columns, na_values=' ?', skipinitialspace=True)

# Convertendo a coluna alvo ('income') para valores binários
data['income'] = data['income'].apply(lambda x: 1 if x == '>50K' else 0)

# Separando variáveis independentes e alvo
X = data.drop('income', axis=1)
y = data['income']

# Identificando colunas numéricas e categóricas
num_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
cat_features = ['workclass', 'education', 'marital-status', 'occupation',
                'relationship', 'race', 'sex', 'native-country']

# Definindo pré-processamento para colunas numéricas e categóricas
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinando transformadores
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ])

# Dividindo o dataset em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Pré-processando os dados
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Aplicando SMOTEENN para balancear as classes no conjunto de treino
smoteenn = SMOTEENN(random_state=42)
X_train_resampled, y_train_resampled = smoteenn.fit_resample(X_train_preprocessed, y_train)

# Salvando as variáveis em um arquivo .pkl
joblib.dump((X_train_resampled, X_test_preprocessed, y_train_resampled, y_test, preprocessor), 'data.pkl')


['data.pkl']

# 2º - Criação e avaliação de modelos


In [ ]:
import joblib
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Carregando dados pré-processados do arquivo .pkl
X_train_resampled, X_test_preprocessed, y_train_resampled, y_test, preprocessor = joblib.load('data.pkl')

# Verificando as colunas do dataset
print("Colunas do X_train_resampled:", X_train_resampled.shape)
print("Colunas do X_test_preprocessed:", X_test_preprocessed.shape)

# Criando um pipeline com apenas o modelo, já que os dados estão pré-processados
pipeline = Pipeline([
    ('model', None)  # O modelo será definido no GridSearchCV
])

# Dicionário de modelos e parâmetros para GridSearch
models = {
    'KNN': (KNeighborsClassifier(), {'model__n_neighbors': [3, 5, 7]}),
    'Decision Tree': (DecisionTreeClassifier(), {'model__max_depth': [3, 5, 7]}),
    'Naive Bayes': (GaussianNB(), {}),
    'SVM': (SVC(probability=True, class_weight='balanced'), {'model__C': [0.1, 1, 10], 'model__gamma': [0.1, 0.01]})
}

# Variáveis para rastrear o melhor modelo
best_f1_score = -1
best_model = None
best_model_name = ''

# Realizando GridSearch para cada modelo
for model_name, (model, params) in models.items():
    print(f"Treinando o modelo: {model_name}")

    # Verifica se o modelo exige dados densos e faz a conversão, se necessário
    if model_name in ['Naive Bayes', 'SVM']:
        # Convertendo dados esparsos para densos
        X_train_resampled_dense = X_train_resampled.toarray()
        X_test_preprocessed_dense = X_test_preprocessed.toarray()
        X_train_to_use = X_train_resampled_dense
        X_test_to_use = X_test_preprocessed_dense
    else:
        X_train_to_use = X_train_resampled
        X_test_to_use = X_test_preprocessed

    # Definindo o modelo no pipeline
    pipeline.set_params(model=model)

    # Configurando o GridSearchCV
    grid_search = GridSearchCV(pipeline, param_grid=params, cv=5, scoring='f1', error_score='raise')

    try:
        # Treinamento do modelo com GridSearch
        grid_search.fit(X_train_to_use, y_train_resampled)

        # Salvando o melhor modelo encontrado
        best_model_filename = f'best_model_{model_name.lower().replace(" ", "_")}.pkl'
        joblib.dump(grid_search.best_estimator_, best_model_filename)
        print(f"Modelo {model_name} salvo em {best_model_filename}")

        # Avaliando o modelo
        y_pred = grid_search.predict(X_test_to_use)
        f1_score = grid_search.best_score_
        print(f"Modelo: {model_name}, Melhor F1: {f1_score}")

        # Verificando se este é o melhor modelo até agora
        if f1_score > best_f1_score:
            best_f1_score = f1_score
            best_model = grid_search.best_estimator_
            best_model_name = model_name

    except ValueError as e:
        print(f"Erro ao treinar o modelo {model_name}: {e}")

# Salvando o modelo com a melhor F1-Score como 'best_model.pkl'
if best_model:
    joblib.dump(best_model, 'best_model.pkl')
    print(f"Melhor modelo ({best_model_name}) salvo como 'best_model.pkl' com F1-Score: {best_f1_score}")
else:
    print("Nenhum modelo foi treinado com sucesso.")

Colunas do X_train_resampled: (29767, 108)
Colunas do X_test_preprocessed: (6513, 108)
Treinando o modelo: KNN


## 3º - Exportação do melhor modelo

In [ ]:
import joblib
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

# Carregando o melhor modelo gerado no treinamento
best_model = joblib.load('best_model.pkl')

# Carregando as variáveis do arquivo .pkl
X_train, X_test, y_train, y_test, preprocessor = joblib.load('data.pkl')

# Avaliando o modelo no conjunto de teste
y_pred = best_model.predict(X_test)
print(f"Classification report para o melhor modelo:")
print(classification_report(y_test, y_pred))
print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"F1 Score: {f1_score(y_test, y_pred)}")
